# Make an SED and calculate fundamental parameters
`SEDkit` is used primarily for the creation and analysis of spectral energy distributions. The code stitches together the provided photometry and spectra and flux calibrates it given a distance. Simple integration then gives the bolometric luminosity. If radius is given or one can be estimated from a given age, effective temperature is also calculated.

This notebook will show you how to make an SED, modify its input parameters, and calculate and plot results.

### Requirements
- `astropy>=3.0.2`
- `bokeh>=0.12.6`
- `astroquery>=0.3.8`

In [48]:
# Imports
from SEDkit import sed, spectrum, synphot
import astropy.units as q
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

## Initialize an SED

Let's create a new SED for an L3.5 brown dwarf. The software assumes a field age if no `age` argument is given.

In [49]:
s = sed.SED(name='2MASSW J0036159+182110')

# Print the metadata
print('\n',s.all_names,'\n\n',s.sky_coords)

Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)

 ['LSPM J0036+1821', '[B2006] J003616.1+182110', 'USNO-B1.0 1083-00010105', '2MUCD 20029', '2MASSW J0036162+182110', '2MASSW J0036159+182110', '2MASS J00361617+1821104'] 

 <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.0674,  18.35290833)>


Given a `name` argument, a SIMBAD query is performed and the alternate names and sky coordinates are extracted.

Alternately, the `sky_coords` argument can be given and the names are similarly retrieved from SIMBAD.

In [50]:
# Clear the names
s.all_names = []

# Add the coordinates
s.sky_coords = 9.0673755*q.deg, 18.352889*q.deg

# Print the metadata
print(s.all_names,'\n\n',s.sky_coords)

['LSPM J0036+1821', '[B2006] J003616.1+182110', 'USNO-B1.0 1083-00010105', '2MUCD 20029', '2MASSW J0036162+182110', '2MASSW J0036159+182110', '2MASS J00361617+1821104'] 

 <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.0673755,  18.352889)>


## Adding a radius and spectral type

Both `radius` and `spectral_type` can be explicitly set upon initialization of the SED or by setting the attributes.

In [51]:
# Set the radius
s.radius = 0.1*q.R_sun, 0.01*q.R_sun

Setting radius to (<Quantity 0.1 solRad>, <Quantity 0.01 solRad>)


Alternatively, if a `spectral_type` is given and `radius` is not, the radius will be estimated from the spectral type according to calculated radius-spectral type relations.

In [52]:
# Clear the radius
s.radius = None

# Set the spectral type
s.spectral_type = 'L3.5'

Setting radius to (<Quantity 0.10127179992838364 solRad>, <Quantity 0.02008219196144978 solRad>)


## Adding photometry
Now let's add some photometry and take a look at how it's stored.

You can pass a CSV file of photometry with the column names 'band', 'magnitude', and 'uncertainty' like so...

In [53]:
# Add photometry from CSV
s.add_photometry_file(resource_filename('SEDkit', 'data/L3_photometry.txt'))
print(s.photometry[['band','app_magnitude','app_magnitude_unc']])

  band   app_magnitude app_magnitude_unc
-------- ------------- -----------------
  SDSS.u        24.688             1.389
  SDSS.g        22.524             0.137
  SDSS.r         19.69             0.019
  SDSS.i        17.179             0.005
  SDSS.z        15.217             0.005
 2MASS.J        12.466             0.027
 2MASS.H        11.588              0.03
2MASS.Ks        11.058             0.021


Or you can add individual measurements by passing the band name, magnitude, and uncertainty to the `add_photometry()` method like so...

In [54]:
# Add individual measurement
s.add_photometry('Johnson.V', 21.43, 0.024)
print(s.photometry[['band','app_magnitude','app_magnitude_unc']])

   band   app_magnitude app_magnitude_unc
--------- ------------- -----------------
   SDSS.u        24.688             1.389
   SDSS.g        22.524             0.137
Johnson.V         21.43             0.024
   SDSS.r         19.69             0.019
   SDSS.i        17.179             0.005
   SDSS.z        15.217             0.005
  2MASS.J        12.466             0.027
  2MASS.H        11.588              0.03
 2MASS.Ks        11.058             0.021


Upper limits can be indicated by passing `None`, `np.nan` or `0` as the uncertainty.

In [55]:
# Add an upper limit
s.add_photometry('Cousins.I', 20.35)
print(s.photometry[['band','app_magnitude','app_magnitude_unc']])

   band   app_magnitude app_magnitude_unc
--------- ------------- -----------------
   SDSS.u        24.688             1.389
   SDSS.g        22.524             0.137
Johnson.V         21.43             0.024
   SDSS.r         19.69             0.019
   SDSS.i        17.179             0.005
Cousins.I         20.35               nan
   SDSS.z        15.217             0.005
  2MASS.J        12.466             0.027
  2MASS.H        11.588              0.03
 2MASS.Ks        11.058             0.021


If the source has a `name` or `sky_coords`, you can query the SDSS, 2MASS, Pan-STARRS, or WISE catalogs with the `find_<catalog>()` method like so...

In [56]:
# Query WISE
s.find_WISE()
print(s.photometry[['band','app_magnitude','app_magnitude_unc']])

   band   app_magnitude app_magnitude_unc
--------- ------------- -----------------
   SDSS.u        24.688             1.389
   SDSS.g        22.524             0.137
Johnson.V         21.43             0.024
   SDSS.r         19.69             0.019
   SDSS.i        17.179             0.005
Cousins.I         20.35               nan
   SDSS.z        15.217             0.005
  2MASS.J        12.466             0.027
  2MASS.H        11.588              0.03
 2MASS.Ks        11.058             0.021
  WISE.W1        10.536             0.024
  WISE.W2        10.245             0.019
  WISE.W3         9.851             0.054
  WISE.W4         8.169               nan


## Adding a distance
A `distance` or `parallax` can be added by setting the attribute with a (value, uncertainty) in `astropy.units.quantitiy.Quantity` units. The other is caluclated from whichever is set first.

In [58]:
# Set the parallax
s.parallax = 114.2*q.mas, 0.8*q.mas

# ...or the distance
# s.distance = 8.87*q.pc, 0.06*q.pc

print(s.parallax, s.distance)

(<Quantity 114.2 mas>, <Quantity 0.8 mas>) (<Quantity 8.76 pc>, <Quantity 0.06 pc>, <Quantity 0.0 pc>)


Alternatively, `SEDkit` can search the Gaia DR2 catalog for a parallax (and a `Gaia.G` magnitude).

In [59]:
s.find_Gaia()
print(s.parallax, s.distance)

(<Quantity 114.4167 mas>, <Quantity 0.20880000293254852 mas>) (<Quantity 8.74 pc>, <Quantity 0.02 pc>, <Quantity 0.0 pc>)


Once a parallax or distance is provided, the data is automatically flux calibrated.

In [60]:
print(s.photometry[['band','app_magnitude','app_magnitude_unc']])

   band   app_magnitude app_magnitude_unc
--------- ------------- -----------------
   SDSS.u        24.688             1.389
   SDSS.g        22.524             0.137
Johnson.V         21.43             0.024
   SDSS.r         19.69             0.019
   Gaia.G       17.5186            0.0017
   SDSS.i        17.179             0.005
Cousins.I         20.35               nan
   SDSS.z        15.217             0.005
  2MASS.J        12.466             0.027
  2MASS.H        11.588              0.03
 2MASS.Ks        11.058             0.021
  WISE.W1        10.536             0.024
  WISE.W2        10.245             0.019
  WISE.W3         9.851             0.054
  WISE.W4         8.169               nan


Once the sky coordinates and distance are set, the dust column density is calculated from the Bayestar17 3D dust map and stored to deredden the photometry.

In [61]:
s.reddening

0.0005679306341335177

## Adding spectra
Spectra aren't necessary to build an SED but let's add some for fun.

Spectra can be added from an ASCII or FITS file...

In [62]:
# Add a spectrum from a file, specifying the astropy units
file = resource_filename('SEDkit', 'data/L3_spectrum.fits')
s.add_spectrum_file(file, wave_units=q.um, flux_units=q.W/q.m**2/q.um)

# Plot it
s.spectra[0].plot(draw=True)

Alternatively, you can add the spectrum manually
```
wave, flux, unc = np.genfromtxt(file, unpack=True)
s.add_spectrum(wave*q.um, flux*q.W/q.m**2/q.um, unc*q.W/q.m**2/q.um)
```

## Calculate results
Now that our SED has all the data it needs, we can call the `results` method, which constructs the SED and caluclates the fundamental parameters.

If no `radius` is given, one will be estimated from model isochrones using the calculated `Lbol` and assumed `age`.

In [63]:
# Calculate and display the results
print(s.results)

    param             value                  unc             units    
------------- ---------------------- ------------------- -------------
         name 2MASSW J0036159+182110                  --            --
          age                    6.0                 4.0           Gyr
     distance                   8.74                0.02            pc
     parallax               114.4167 0.20880000293254852           mas
       radius    0.10127179992838364 0.02008219196144978        solRad
          SpT                   L3.5                  --            --
spectral_type                   73.5                 0.5            --
   membership                     --                  --            --
         fbol               5.03e-11            1.99e-13 erg / (cm2 s)
         mbol                 14.264               0.004            --
         Lbol                4.6e+29            2.78e+27       erg / s
     Lbol_sun                  -3.92               0.003            --
      

Cool. Filippazzo et al. (2015) has this object at $1869 \pm 64$ K.

We can also fit a blackbody to the data using a Levenberg-Marquardt least-squares minimization. The calculated `Teff` will be used as the initial guess if available. Otherwise, this can be set manually with the `Teff_init` argument.

In [64]:
# Run the blackbody fit
s.fit_blackbody()


Blackbody fit: 1698 K


We can also fit the spectrum to estimate a spectral type.

In [65]:
# Run the spectral type fit
# s.fit_spectral_type()

In [66]:
# Plot the SED
fig = s.plot(integral=True)

## Fine tuning
A nice feature of `SEDkit` is that the caluclations cascade when you change an attribute.

For example, the radius esitmated from Filippazzo et al. (2015) is 0.101 Solar radii. Let's say we want to see what the results are with a slightly smaller radius. We can just set the attribute (value, uncertainty) and show the results again.

In [67]:
s.radius = 0.9*q.Rjup, 0.1*q.Rjup
print(s.results)

Setting radius to (<Quantity 0.9 jupiterRad>, <Quantity 0.1 jupiterRad>)
    param             value                  unc             units    
------------- ---------------------- ------------------- -------------
         name 2MASSW J0036159+182110                  --            --
          age                    6.0                 4.0           Gyr
     distance                   8.74                0.02            pc
     parallax               114.4167 0.20880000293254852           mas
       radius                    0.9                 0.1    jupiterRad
          SpT                   L3.5                  --            --
spectral_type                   73.5                 0.5            --
   membership                     --                  --            --
         fbol               5.03e-11            1.99e-13 erg / (cm2 s)
         mbol                 14.264               0.004            --
         Lbol                4.6e+29            2.78e+27       erg / s
    

So the Teff went from 1898K to 1987K, which is right since if we assume a smaller radius with the same Lbol, the Teff should be higher.

## Export the SED
Now we can export the result to a zip file (if `zipped=True`) or directory like so:

In [ ]:
s.export('/Users/jfilippazzo/Desktop/', zipped=True)